In [75]:
import nltk

In [76]:
import pandas as pd
import numpy as np

In [77]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [78]:
from collections import Counter

In [79]:
from imblearn.under_sampling import RandomUnderSampler

# NLP Pipeline 

![](images/pipeline-walkthrough1.png)

Below is a to do list when converting text into vector form: 

**Clean text and Create a Bag of Words (BoW)**
>1. Lowercase the text
2. Tokenize 
3. Strip out punctuation or undesirable text
4. Remove Stopwords 
5. Stemming or Lemmatizing
6. Compute N-Grams
7. Use this to create BoW

**Vectorize BoW**
>8. Term Frequencies
9. Document Frequencies
10. TF-IDF
11. Normalize vectors

Let's go through both what each of these steps are and how to do them in python with the following corpus of comments about data science...
 

In [80]:
#funtion to cut off extra intro paragraphs from beautiful soup scrape

# def trim_fat(string):
#     return string[35:-115]

In [81]:
# function to vectorize the type_of_material series into a y target vector.
def vectorize_type(ser):
    y = ser.copy()
    y.replace({'Op-Ed': 1,'News': 0}, inplace=True)
    return y

In [82]:
#rate of correct predictions out of total predictions
def metrics_(tn, fp, fn, tp):
    accuracy = (tp + tn) / (tn + fn + tp + fp)
    print(f'accuracy = {accuracy}')
    recall = (tp) / (tp + fn)
    print(f'recall = {recall}')
    precision = (tp) / (tp + fp)
    print(f'precision = {precision}')

In [83]:
%%time
_2019 = pd.read_csv('data/trim2019_text_type.csv', index_col='Unnamed: 0')

CPU times: user 1.62 s, sys: 260 ms, total: 1.88 s
Wall time: 1.9 s


In [84]:
%%time
_2019_df = _2019.copy()

CPU times: user 1.41 ms, sys: 21 µs, total: 1.43 ms
Wall time: 1.44 ms


In [85]:
#define X. X is currently pandas series of unsplit strings

X = _2019_df.text

In [86]:
type(X[0])

str

In [87]:
type(X)

pandas.core.series.Series

In [88]:
# define y as a series of op-ed or news

y = _2019_df.type_of_material

In [89]:
%%time
# vectorize y in to (1, 0) (op-ed, news)

y = vectorize_type(y)

CPU times: user 12.4 ms, sys: 1.85 ms, total: 14.3 ms
Wall time: 13.9 ms


In [90]:
#turn series into list...

corpus = list(X)

# sklearn TfidfVectorizer(stop_words='english', strip_accents='ascii')

In [91]:
%%time
#create vectorizer

vectorizer = TfidfVectorizer(#input='content', 
#                 encoding='utf-8', 
#                 decode_error='strict', 
                 strip_accents=None, 
                 lowercase=True, 
#                 preprocessor=None, 
#                 tokenizer=None, 
#                 analyzer='word', 
                 stop_words='english', 
#                 token_pattern='(?u)\b\w\w+\b', 
#                 ngram_range=(1, 1), 
#                 max_df=1.0, 
#                 min_df=1, 
                 max_features=None, 
#                 vocabulary=None, 
#                 binary=False, 
#                 dtype=<class 'numpy.float64'>, 
#                 norm='l2', 
#                 use_idf=True, 
#                 smooth_idf=True, 
#                 sublinear_tf=False
)
X = vectorizer.fit_transform(corpus)

CPU times: user 27.2 s, sys: 672 ms, total: 27.8 s
Wall time: 28.1 s


In [92]:
%%time
feature_names = vectorizer.get_feature_names()

CPU times: user 197 ms, sys: 5.26 ms, total: 202 ms
Wall time: 203 ms


In [93]:
%%time
stop_words = vectorizer.get_stop_words()

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 9.78 µs


In [94]:
feature_names;

In [95]:
len(feature_names)

215840

In [96]:
len(stop_words)

318

# resample class size w/ imbalanced learn

In [97]:
y.shape

(41748,)

In [98]:
X.shape

(41748, 215840)

In [99]:
%%time
#balance the classes

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

CPU times: user 29 µs, sys: 63 µs, total: 92 µs
Wall time: 95.8 µs


In [100]:
%%time
#X, y --> X_resampled, y_resampled
X_resampled, y_resampled = rus.fit_resample(X, y)


#return a list of tuples for item, and count of item. in this case 4139 each
print(sorted(Counter(y_resampled).items()))

[(0, 4139), (1, 4139)]
CPU times: user 31.1 ms, sys: 16 ms, total: 47.2 ms
Wall time: 47.2 ms


In [101]:
y_resampled.shape

(8278,)

In [102]:
X_resampled.shape

(8278, 215840)

In [103]:
%%time
#test, train, split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

CPU times: user 12.9 ms, sys: 13.7 ms, total: 26.6 ms
Wall time: 25.7 ms


# Multinomial Naive Bayes/imbalanced learn/TFIDF vectorizer

In [104]:
# %%time

# multinm_clf = MultinomialNB()
# multinm_clf.fit(X_train, y_train)

In [105]:
#multinm_clf.class_count_

In [106]:
#y_pred = multinm_clf.predict(X_test)

In [107]:
# #clf.score = accuracy = 'true'(pos/neg) / total

# multinm_clf.score(X_test, y_test)

In [108]:
#confusion_matrix(y_true = y_test, y_pred = y_pred)

In [109]:
# tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
# (tn, fp, fn, tp)

In [110]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# accuracy

In [111]:
# # rate of actual op-ed articles, out of all the actual od-ed articles

# recall = (tp) / (tp + fn)
# recall

In [112]:
# # rate of correct predictions of op-ed articles out of all predictions

# precision = (tp) / (tp + fp)
# precision

In [113]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# print(f'accuracy = {accuracy}')
# recall = (tp) / (tp + fn)
# print(f'recall = {recall}')
# precision = (tp) / (tp + fp)
# print(f'precision = {precision}')

# random forest classifier

In [114]:
# class sklearn.ensemble.RandomForestClassifier(n_estimators=100, 
#                                               *, 
#                                               criterion='gini', 
#                                               max_depth=None, 
#                                               min_samples_split=2, 
#                                               min_samples_leaf=1, 
#                                               min_weight_fraction_leaf=0.0, 
#                                               max_features='auto', 
#                                               max_leaf_nodes=None, 
#                                               min_impurity_decrease=0.0, 
#                                               min_impurity_split=None, 
#                                               bootstrap=True, 
#                                               oob_score=False, 
#                                               n_jobs=None, 
#                                               random_state=None, 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               class_weight=None, 
#                                               ccp_alpha=0.0, 
#                                               max_samples=None)

In [115]:
%%time

rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

CPU times: user 39 µs, sys: 1 µs, total: 40 µs
Wall time: 42 µs


In [116]:
%%time

rf_clf.fit(X_train, y_train)

CPU times: user 401 ms, sys: 18.1 ms, total: 419 ms
Wall time: 426 ms


RandomForestClassifier(max_depth=2, random_state=0)

In [117]:
y_pred = rf_clf.predict(X_test)

In [118]:
#clf.score = accuracy = 'true'(pos/neg) / total

rf_clf.score(X_test, y_test)

0.9415458937198068

In [119]:
confusion_matrix(y_true = y_test, y_pred = y_pred)

array([[1003,   17],
       [ 104,  946]])

In [120]:
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
(tn, fp, fn, tp)

(1003, 17, 104, 946)

In [121]:
# #rate of correct predictions out of total predictions

# accuracy = (tp + tn) / (tn + fn + tp + fp)
# accuracy

In [122]:
# # rate of actual op-ed articles, out of all the actual od-ed articles

# recall = (tp) / (tp + fn)
# recall

In [123]:
# # rate of correct predictions of op-ed articles out of all predictions

# precision = (tp) / (tp + fp)
# precision

In [124]:
#rate of correct predictions out of total predictions

accuracy = (tp + tn) / (tn + fn + tp + fp)
print(f'accuracy = {accuracy}')
recall = (tp) / (tp + fn)
print(f'recall = {recall}')
precision = (tp) / (tp + fp)
print(f'precision = {precision}')

accuracy = 0.9415458937198068
recall = 0.900952380952381
precision = 0.9823468328141225


In [125]:
# class sklearn.ensemble.RandomForestClassifier(n_estimators=100, 
#                                               *, 
#                                               criterion='gini', 
#                                               max_depth=None, 
#                                               min_samples_split=2, 
#                                               min_samples_leaf=1, 
#                                               min_weight_fraction_leaf=0.0, 
#                                               max_features='auto', 
#                                               max_leaf_nodes=None, 
#                                               min_impurity_decrease=0.0, 
#                                               min_impurity_split=None, 
#                                               bootstrap=True, 
#                                               oob_score=False, 
#                                               n_jobs=None, 
#                                               random_state=None, 
#                                               verbose=0, 
#                                               warm_start=False, 
#                                               class_weight=None, 
#                                               ccp_alpha=0.0, 
#                                               max_samples=None)

# NLTK 

In [126]:
nltk_2019_df = _2019.copy()

In [127]:
#define X. X is currently pandas series of unsplit strings

X_nltk = nltk_2019_df.text

In [128]:
# define y as a series of op-ed or news

y_nktk = nltk_2019_df.type_of_material

In [129]:
%%time
# vectorize y in to (1, 0) (op-ed, news)

y_nltk = vectorize_type(y_nktk)

CPU times: user 12.5 ms, sys: 2.88 ms, total: 15.4 ms
Wall time: 14.5 ms


In [130]:
#turn series into list...

corpus_nltk = list(X_nltk)

### tokenize w/ nltk

In [131]:
%%time
from nltk.tokenize import RegexpTokenizer
regex_tokenizer = nltk.RegexpTokenizer(r"\w+")
tokenized_punc = [regex_tokenizer.tokenize(article.lower())for article in corpus_nltk]

CPU times: user 15.5 s, sys: 872 ms, total: 16.4 s
Wall time: 16.6 s


In [132]:
(len(tokenized_punc[0])) #2218

2218

In [133]:
len(tokenized_punc)

41748

In [134]:
from nltk.tokenize import word_tokenize

In [135]:
type(word_tokenize)

function

In [136]:
# %%time
# tokenized = [word_tokenize(article.lower()) for article in corpus_nltk]

# CPU times: user 4min 46s, sys: 2.39 s, total: 4min 48s
# Wall time: 4min 50s

In [137]:
# #list of list of strings. one list of strings per documents. list are various lengths around 1000

# len(tokenized[0]) #2596

In [138]:
from nltk.corpus import stopwords

## take out stop work via ntlk. does this work against sklearn when i vectorize

In [139]:
%%time
stop = set(stopwords.words('english'))
tokenized_docs = [[word for word in words if word not in stop]
            for words in tokenized_punc]

CPU times: user 4.77 s, sys: 270 ms, total: 5.04 s
Wall time: 5.1 s


In [140]:
# #hopefully this reduced the number of strings / list

# len(tokenized[1])

In [141]:
# #docs is new tokenized, but with stop words removed

# len(tokenized_docs)

# stemming/lemmatization

In [142]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [143]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

In [144]:
# %%time
# docs_porter = [[porter.stem(word) for word in words]
#                    for words in docs]
# docs_snowball = [[snowball.stem(word) for word in words]
#                      for words in docs]
# docs_wordnet = [[wordnet.lemmatize(word) for word in words]
#                     for words in docs]

# CPU times: user 14min 59s, sys: 18.4 s, total: 15min 18s
# Wall time: 15min 26s

In [145]:
# %%time
# docs_porter = [[porter.stem(word) for word in words]
#                    for words in docs]

# CPU times: user 7min 16s, sys: 5.21 s, total: 7min 21s
# Wall time: 7min 22s

In [151]:
# %%time
# snowball_stemm = [[snowball.stem(word) for word in words]
#                      for words in tokenized_docs]

# # CPU times: user 5min 5s, sys: 5.98 s, total: 5min 11s
# # Wall time: 5min 13s

In [152]:
# %%time
# docs_wordnet = [[wordnet.lemmatize(word) for word in words]
#                     for words in docs]

# CPU times: user 1min 24s, sys: 4.5 s, total: 1min 28s
# Wall time: 1min 30s

In [153]:
# %%time
# ## Print the stemmed and lemmatized words from the first document
# print("%16s %16s %16s %16s" % ("word", "porter", "snowball", "lemmatizer"))
# for i in range(min(len(docs_porter[0]), len(docs_snowball[0]), len(docs_wordnet[0]))):
#     p, s, w = docs_porter[0][i], docs_snowball[0][i], docs_wordnet[0][i]
#     if len(set((p, s, w))) != 1:
#         print("%16s %16s %16s %16s" % (docs[0][i], p, s, w))
#         print(docs[0][i], w)


In [154]:
#docs and lemmatizer are the same?

# I choose SNOWBALL!!!! to sklearn

In [158]:
type(corpus_nltk)

list

In [159]:
type(corpus_nltk[0])

str

In [160]:
# snowball = SnowballStemmer('english')
# snowball_tokenized = [snowball.stem(word) for word in word_tokenize(doc.lower())]

In [175]:
%%time
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

def snowball_tokenize(doc):
    snowball = SnowballStemmer('english')
    return [snowball.stem(word) for word in word_tokenize(doc.lower())]

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.1 µs


In [ ]:
%%time
#create vectorizer

vectorizer = TfidfVectorizer(#input='content', 
#                 encoding='utf-8', 
#                 decode_error='strict', 
                 strip_accents='ascii', 
                 lowercase=True, 
#                 preprocessor=None, 
                 tokenizer=snowball_tokenize, 
#                 analyzer='word', 
                 stop_words='english', 
#                 token_pattern='(?u)\b\w\w+\b', 
#                 ngram_range=(1, 1), 
#                 max_df=1.0, 
#                 min_df=1, 
                 max_features=None, 
#                 vocabulary=None, 
#                 binary=False, 
#                 dtype=<class 'numpy.float64'>, 
#                 norm='l2', 
#                 use_idf=True, 
#                 smooth_idf=True, 
#                 sublinear_tf=False
)



In [ ]:
%%time
X_snowball = vectorizer.fit(corpus_nltk)

# CPU times: user 12min 37s, sys: 2.61 s, total: 12min 39s
# Wall time: 12min 31s w/ 10,000 features

In [ ]:
%%time
X_snowball = X_snowball.transform(corpus_nltk)

In [ ]:
#class sklearn.feature_extraction.text.CountVectorizer(*, 
#                                                       input='content', 
#                                                       encoding='utf-8', 
#                                                       decode_error='strict', 
#                                                       strip_accents=None, 
#                                                       lowercase=True, 
#                                                       preprocessor=None, 
#                                                       tokenizer=None, 
#                                                       stop_words=None, 
#                                                       token_pattern='(?u)\b\w\w+\b', 
#                                                       ngram_range=(1, 1), 
#                                                       analyzer='word', 
#                                                       max_df=1.0, 
#                                                       min_df=1, 
#                                                       max_features=None, 
#                                                       vocabulary=None, 
#                                                       binary=False, 
#                                                       dtype=<class 'numpy.int64'>
#)

In [ ]:
count_vectorizer = CountVectorizer(strip_accents='None',
                                   lowercase=True,
                                   tokenizer=snowball_tokenize,
                                   stop_words='english',
                                   max_features=None)

In [ ]:
%%time
#balance the classes

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)

In [ ]:
%%time
#X, y --> X_resampled, y_resampled
X_resampled, y_resampled = rus.fit_resample(X_snowball, y_nltk)
print(sorted(Counter(y_resampled).items()))

In [ ]:
%%time
#test, train, split

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=50)

In [ ]:
%%time

rf_clf = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
%%time

rf_clf.fit(X_train, y_train)

In [ ]:
y_pred = rf_clf.predict(X_test)

In [ ]:
#clf.score = accuracy = 'true'(pos/neg) / total

rf_clf.score(X_test, y_test)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true = y_test, y_pred = y_pred).ravel()
(tn, fp, fn, tp)
metrics_(tn, fp, fn, tp)
print(f'tn={tn}, fp={fp}, fn={fn}, tp={tp})')

In [ ]:
nltk_features = vectorizer.get_feature_names()

In [ ]:
nltk_stop = vectorizer.get_stop_words()

In [ ]:
type(nltk_features)

In [ ]:
len(nltk_stop)

In [ ]:
X_snowball.shape

In [ ]:
# visualizing the bag of words
columns = nltk_features

In [ ]:
feature_import = rf_clf.feature_importances_

In [ ]:
# property feature_importances_
# The impurity-based feature importances.

# The higher, the more important the feature. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.

# Warning: impurity-based feature importances can be misleading for high cardinality features (many unique values). See sklearn.inspection.permutation_importance as an alternative.

# Returns
# feature_importances_ndarray of shape (n_features,)
# The values of this array sum to 1, unless all trees are single node trees consisting of only the root node, in which case it will be an array of zeros.



In [ ]:
feat_scores = pd.Series(feature_import,
                           index=nltk_features)
feat_scores = feat_scores.sort_values()
ax = feat_scores.plot(kind='barh', figsize=(10,20))
ax.set_title('"Decrease in Impurity" Importance')
ax.set_xlabel('Average contribution to the reduction in variance');

In [ ]:
r = permutation_importance(rf_clf, X_test, y_test, n_repeats=30, random_state=0)
for index in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{feature_names[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")